In [36]:
from IPython import display
display.clear_output()
import ultralytics
ultralytics.checks()
from ultralytics import YOLO
import cv2
import os
import re
import glob

Ultralytics 8.3.49  Python-3.10.11 torch-2.4.1+cu118 CUDA:0 (Quadro T1000 with Max-Q Design, 4096MiB)
Setup complete  (12 CPUs, 15.8 GB RAM, 418.0/471.6 GB disk)


Convert video into a folder of jpg frames

In [7]:
def convert_mp4_to_frames(video_path, output_folder):
    """
    Converts an MP4 video into a folder of JPG frames.

    Args:
        video_path (str): Path to the input MP4 video.
        output_folder (str): Path to the folder where frames will be saved.
    """
    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Open the video file
    video_capture = cv2.VideoCapture(video_path)

    if not video_capture.isOpened():
        print(f"Error: Cannot open video file {video_path}")
        return

    frame_count = 0

    while True:
        ret, frame = video_capture.read()

        # Break the loop when no frames are left to read
        if not ret:
            break

        # Define the frame's output filename
        frame_filename = os.path.join(output_folder, f"{frame_count:04d}.jpg")

        # Save the current frame as a JPG file
        cv2.imwrite(frame_filename, frame)
        print(f"Saved frame {frame_count} to {frame_filename}")

        frame_count += 1

    # Release the video capture object
    video_capture.release()
    print(f"Conversion complete! Total frames extracted: {frame_count}")

# Example usage:
video_path = "../short_sample_nl_match.mp4"
output_folder = "../hybrid_workflow/originalseq"
convert_mp4_to_frames(video_path, output_folder)

Saved frame 0 to ../hybrid_workflow/originalseq\0000.jpg
Saved frame 1 to ../hybrid_workflow/originalseq\0001.jpg
Saved frame 2 to ../hybrid_workflow/originalseq\0002.jpg
Saved frame 3 to ../hybrid_workflow/originalseq\0003.jpg
Saved frame 4 to ../hybrid_workflow/originalseq\0004.jpg
Saved frame 5 to ../hybrid_workflow/originalseq\0005.jpg
Saved frame 6 to ../hybrid_workflow/originalseq\0006.jpg
Saved frame 7 to ../hybrid_workflow/originalseq\0007.jpg
Saved frame 8 to ../hybrid_workflow/originalseq\0008.jpg
Saved frame 9 to ../hybrid_workflow/originalseq\0009.jpg
Saved frame 10 to ../hybrid_workflow/originalseq\0010.jpg
Saved frame 11 to ../hybrid_workflow/originalseq\0011.jpg
Saved frame 12 to ../hybrid_workflow/originalseq\0012.jpg
Saved frame 13 to ../hybrid_workflow/originalseq\0013.jpg
Saved frame 14 to ../hybrid_workflow/originalseq\0014.jpg
Saved frame 15 to ../hybrid_workflow/originalseq\0015.jpg
Saved frame 16 to ../hybrid_workflow/originalseq\0016.jpg
Saved frame 17 to ../hyb

In [12]:
model = YOLO("C:/Studie/BEP/BEP_Football_tracking/YOLO_results/Ball10/best.pt")

In [14]:
# Run inference on 'bus.jpg' with arguments
results = model.predict("C:/Studie/BEP/BEP_Football_tracking/sample_nl_match.mp4", save=False, imgsz=1280, conf=0.60, show=True, save_txt= True, max_det = 1, project = 'C:/Studie/BEP/BEP_Football_tracking/hybrid_workflow/YOLO_ball')



WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/727) C:\Studie\BEP\BEP_Football_tracking\sample_nl_match.mp4: 736x1280 (no detections), 152.6ms
video 1/1 (frame 2/727) C:\Studie\BEP\BEP_Football_tracking\sample_nl_match.mp4: 736x1280 (no detections), 86.8ms
video 1/1 (frame 3/727) C:\Studie\BEP\BEP_Football_tracking\sample_nl_match.mp4: 736x1280 (no detections), 77.8ms
video 1/1 (frame 4/727) C:\Studie\BEP\BEP_Football_tracking\sample_nl_match.mp4: 736x1280 (no detections), 46.9ms
v

Code that gets rid of first variable in yolo text predictions so it is ready for prompting(assuming a 1920 by 1080 resolution
YOLO uses a center based relative format x,y(of center) width hight between 0 and 1
Samurai uses as promp (x,y of the top left, widht height) all in pixels

In [30]:
input_txt = 'C:\Studie\BEP\BEP_Football_tracking\hybrid_workflow\YOLO_ball\predict2\labels\sample_nl_match_18.txt'
output_folder = r'C:\Studie\BEP\BEP_Football_tracking\hybrid_workflow\prompts'

In [31]:
def convert_filename(yolo_file):
    # Use a regular expression to find the number at the end of the filename, before the extension
    match = re.search(r'(\d+)(?=\.[a-zA-Z]+$)', yolo_file)

    if match:
        # Extract the number (it can have multiple digits)
        number = match.group(1)

        # Return the new filename using only the extracted number and .txt extension
        return f"{number}.txt"
    else:
        raise ValueError("Filename does not have a number at the end")

def yolo_to_samurai(yolo_file, output_folder, resolution=(1920, 1080)):
    # Define the target directory
    target_directory = output_folder

    # Convert YOLO filename to Samurai format filename
    output_filename = convert_filename(yolo_file)

    # Combine the target directory and the new filename to create the full path
    output_file = os.path.join(target_directory, output_filename)

    # Ensure the target directory exists
    os.makedirs(target_directory, exist_ok=True)

    with open(yolo_file, 'r') as file:
        yolo_lines = file.readlines()

    samurai_lines = []
    for line in yolo_lines:
        # Split YOLO line into components
        values = line.split()
        if len(values) != 5:
            continue  # Skip malformed lines

        _, x_center_rel, y_center_rel, width_rel, height_rel = map(float, values)

        # Convert relative values to pixel values
        x_center = x_center_rel * resolution[0]
        y_center = y_center_rel * resolution[1]
        width = width_rel * resolution[0]
        height = height_rel * resolution[1]

        # Calculate top-left corner coordinates
        x_top_left = x_center - width / 2
        y_top_left = y_center - height / 2

        # Format as Samurai bounding box
        samurai_line = f"{int(round(x_top_left))}, {int(round(y_top_left))}, {int(round(width))}, {int(round(height))}\n"
        samurai_lines.append(samurai_line)

    # Write Samurai format to the specified output file path
    with open(output_file, 'w') as file:
        file.writelines(samurai_lines)


yolo_to_samurai(input_txt, output_folder)

In [34]:
# Function to process all txt files in a directory
def process_all_files(input_folder, output_folder):
    # Find all .txt files in the directory
    yolo_files = glob.glob(os.path.join(input_folder, "*.txt"))

    for yolo_file in yolo_files:
        yolo_to_samurai(yolo_file, output_folder)



In [37]:
# Example usage
input_folder = 'C:\Studie\BEP\BEP_Football_tracking\hybrid_workflow\YOLO_ball\predict2\labels'
output_folder = 'C:\Studie\BEP\BEP_Football_tracking\hybrid_workflow\prompts'

process_all_files(input_folder, output_folder)

Initialization of a count variable that counts the current frame we are on, then puts all standard frames in 